In [1]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/tlp/synthetic-tvaes.csv')

In [3]:
df.head()

,etiq-diagExpTLPtext,etiq-diagExpTLPcode,etiq-diagExpTLPtext_R2,etiq-diagExpTLPcode_R2,demo-genero,demo-fechaEvaluacion,demo-fechaNacimiento,demo-rangoEdad,demo-edad,clin-reservaCognitiva_total,...,eval-TLP-Tavec-25_Discriminabilidad-PD,eval-TLP-Tavec-26_Sesgo-PD,eval-TLP-Tavec-27_RI_B_frente_RI_A1-PD,eval-TLP-Tavec-28_RL_CP_frente_RI_A5-PD,eval-TLP-Tavec-29_RCl_Cp_frente_RCl_LP-PD,eval-TLP-Tavec-30_RL_LP_frente_RL_CP-PD,eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD,eval-TLP-Tavec-32_Recon_Ac_frente_RL_LP-PD,eval-TLP-Tavec-33_Recon_Ac_frente_RCl_LP-PD,ED_2Clases
0,"DC MUY LEVE, ejecutivo - atencional",2.07,"DC MUY LEVE, ejecutivo - atencional",2.07,2,2024-02-22T00:00:00,1942-05-08T00:00:00,Rango3,81,12,...,84.09,0.14,-20.00,0.00,33.33,37.50,-33.33,-38.46,-7.69,D
1,"DC LEVE, ejecutivo - atencional",3.07,"DC LEVE, multidominio amnésico",3.06,2,2024-06-26T00:00:00,1948-10-07T00:00:00,Rango3,75,8,...,79.55,-0.33,-71.43,12.50,0.00,0.00,0.00,-20.00,-20.00,D
2,NORMAL,1.00,"DC MUY LEVE, multidominio amnésico",2.06,2,2024-05-14T00:00:00,1952-05-08T00:00:00,Rango2,72,11,...,88.64,-0.60,-14.29,75.00,25.00,-20.00,0.00,-16.67,-16.67,H
3,"DC MUY LEVE, amnésico",2.01,"DC MUY LEVE, amnésico",2.01,2,2024-04-10T00:00:00,1964-01-27T00:00:00,Rango1,60,18,...,90.91,0.60,60.00,16.67,0.00,50.00,-20.00,-50.00,-37.50,D
4,"DC MUY LEVE, multidominio amnésico",2.06,"DC LEVE, multidominio amnésico",3.06,2,2024-05-14T00:00:00,1959-09-16T00:00:00,Rango1,64,13,...,86.36,-0.71,0.00,66.67,0.00,-33.33,0.00,-10.00,-10.00,D


In [4]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((382, 84), (96, 84))

In [5]:
predictor= TabularPredictor(
    label='ED_2Clases',
    problem_type='binary',
    eval_metric='roc_auc',
    sample_weight='balance_weight',
    path='AutogluonModels/synthetic/tvaes/v1'
    )

In [6]:
predictor.fit(
    train_data = df_train,
    presets = ['high_quality'],
    time_limit = 2 * 3600,
    auto_stack = True,
    excluded_model_types=['KNN','RF','XT', 'LR'],
    verbosity = 2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       7.66 GB / 15.94 GB (48.1%)
Disk Space Avail:   118.78 GB / 446.36 GB (26.6%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and

(_ray_fit pid=8592) [1000]	valid_set's binary_logloss: 3.69095e-05


(_dystack pid=7812) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=7812) 	2.24s	 = Training   runtime
(_dystack pid=7812) 	0.07s	 = Validation runtime
(_dystack pid=7812) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 97.81s of the 696.38s of remaining time.
(_dystack pid=7812) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.06%)
(_ray_fit pid=13996) No improvement since epoch 4: early stopping
(_dystack pid=7812) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=7812) 	7.32s	 = Training   runtime
(_dystack pid=7812) 	0.13s	 = Validation runtime
(_dystack pid=7812) Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 86.89s of the 685.45s of remaining time.
(_dystack pid=7812) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=7.52%)
(_ray_fit pid=2704) 	Ran out of time, early stopping on itera

(_ray_fit pid=13396) [1000]	valid_set's binary_logloss: 0.0043799 [repeated 8x across cluster]


(_dystack pid=7812) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=7812) 	4.18s	 = Training   runtime
(_dystack pid=7812) 	0.04s	 = Validation runtime
(_dystack pid=7812) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 5.60s of the 604.16s of remaining time.
(_dystack pid=7812) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.03%)
(_ray_fit pid=6760) 	Ran out of time, early stopping on iteration 436. [repeated 7x across cluster]
(_dystack pid=7812) 	Warning: Exception caused NeuralNetTorch_r22_BAG_L1 to fail during training... Skipping this model.
(_dystack pid=7812) 		ray::_ray_fit() (pid=1352, ip=127.0.0.1)
(_dystack pid=7812)   File "python\ray\_raylet.pyx", line 1889, in ray._raylet.execute_task
(_dystack pid=7812)   File "c:\Users\jgala\uned\tfm\synthetic-data\.venv\lib\site-packages\autogluon\core\models\ensemble\fold_fitting_strategy.py", line 413, in _ray_fit
(_dystack pid=7

In [7]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                               model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                    LightGBM_BAG_L1   1.000000     roc_auc       0.029233     1.798387                0.029233           1.798387            1      False          2
1                WeightedEnsemble_L3   1.000000     roc_auc       0.052571     1.097105                0.000000           0.027211            3      False         32
2                  LightGBMXT_BAG_L1   1.000000     roc_auc       0.052571     1.069894                0.052571           1.069894            1      False          1
3                LightGBM_r96_BAG_L1   1.000000     roc_auc       0.053207     4.919226                0.053207           4.919226            1      False         11
4                WeightedEnsemble_L2   1.000000     roc_auc       0.053573     1.118017                0.001

{'model_types': {'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r9_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r96_BAG_L1': 'StackerEnsembleModel_LGB',
  'XGBoost_r33_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'CatBoost_r137_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_r102_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r13_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r188_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r145_BAG_L1': 'StackerEnsembleModel_NNFast

In [8]:
auxiliary_metrics = ['balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'roc_auc', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [9]:
predictor.evaluate(
    df_test,
    silent = False,
    auxiliary_metrics = auxiliary_metrics
    )

Evaluation: roc_auc on test data: 0.9999999999999999
Evaluations on test data:
{
    "roc_auc": 0.9999999999999999,
    "accuracy": 1.0,
    "balanced_accuracy": 1.0,
    "mcc": 1.0,
    "f1": 1.0,
    "precision": 1.0,
    "recall": 1.0
}


{'roc_auc': 0.9999999999999999,
 'accuracy': 1.0,
 'balanced_accuracy': 1.0,
 'mcc': 1.0,
 'f1': 1.0,
 'precision': 1.0,
 'recall': 1.0}

In [12]:
predictors = predictor.leaderboard(
    df_train,
    extra_metrics = auxiliary_metrics,
    extra_info=True,
    silent=True
)

In [13]:
predictors

,model,score_test,balanced_accuracy,f1,f1_macro,f1_micro,roc_auc,average_precision,precision,recall,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,LightGBM_BAG_L1_FULL,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.15, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[eval-TLP-Tavec-24_FP-PD, clin-ansiedad, eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD, clin-cardiopatiaIsquemica, __nlp__.atencional dc, eval-TLP-FigRey-totalCopia-PD, eval-TLP-Stroop-palabrasColor-PD, eval-TLP-Tavec-1_RI_A1-PD, etiq-diagExpTLPtext.char_count, __nlp__._total_, clin-tipoAlcohol.capital_ratio, clin-Hipercolesterolemia, clin-tipoAlcohol.lower_ratio, eval-TLP-FigRey-totalMemoria-PD, ques-Sus-total-PD, __nlp__.amnésico dc, eval-TLP-Tavec-4_RI_B-PD, clin-tipoAlcohol.word_count, demo-genero, etiq-diagExpTLPtext.symbol_ratio.-, eval-TLP-FigRey-Duracion-PD, etiq-diagExpTLPtext.word_cou...",None,"{'learning_rate': 0.05, 'num_boost_round': 221}",{'num_boost_round': 221},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}",[],[]
1,XGBoost_BAG_L1_FULL,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.15, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[eval-TLP-Tavec-24_FP-PD, clin-ansiedad, eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD, clin-cardiopatiaIsquemica, __nlp__.atencional dc, eval-TLP-FigRey-totalCopia-PD, eval-TLP-Stroop-palabrasColor-PD, eval-TLP-Tavec-1_RI_A1-PD, etiq-diagExpTLPtext.char_count, __nlp__._total_, clin-tipoAlcohol.capital_ratio, clin-Hipercolesterolemia, clin-tipoAlcohol.lower_ratio, eval-TLP-FigRey-totalMemoria-PD, ques-Sus-total-PD, __nlp__.amnésico dc, eval-TLP-Tavec-4_RI_B-PD, clin-tipoAlcohol.word_count, demo-genero, etiq-diagExpTLPtext.symbol_ratio.-, eval-TLP-FigRey-Duracion-PD, etiq-diagExpTLPtext.word_cou...",None,"{'n_estimators': 105, 'learning_rate': 0.1, 'n_jobs': -1, 'proc.max_category_levels': 100, 'objective': 'binary:logistic', 'booster': 'gbtree'}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}",[],"[CatBoost_r9_BAG_L2_FULL, LightGBMXT_BAG_L2_FULL, CatBoost_r177_BAG_L2_FULL, LightGBM_r131_BAG_L2_FULL, CatBoost_BAG_L2_FULL, XGBoost_BAG_L2_FULL, LightGBMLarge_BAG_L2_FULL, LightGBM_r96_BAG_L2_FULL, CatBoost_r137_BAG_L2_FULL, NeuralNetFastAI_BAG_L2_FULL, NeuralNetFastAI_r191_BAG_L2_FULL, XGBoost_r33_BAG_L2_FULL, LightGBM_BAG_L2_FULL]"
2,LightGBMXT_BAG_L1_FULL,1.0,1.0,

In [15]:
predictor.feature_importance(data=df_train)

These features in provided data are not utilized by the predictor and will be ignored: ['clin-Covid_sintomaSarpullido', 'ques-MMSE-fijacion-PD']
Computing feature importance via permutation shuffling for 81 features using 382 rows with 5 shuffle sets...
	25.67s	= Expected runtime (5.13s per shuffle set)
	6.21s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
etiq-diagExpTLPcode,0.272791,0.025795,0.000009,5,0.325903,0.219678
etiq-diagExpTLPtext,0.003265,0.002156,0.013821,5,0.007705,-0.001175
eval-TLP-Tavec-32_Recon_Ac_frente_RL_LP-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
eval-TLP-Tavec-3_RI_AT-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
eval-TLP-Tavec-10_RL_LP-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
...,...,...,...,...,...,...
ques-NavEspacial-All-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
ques-QuejasMemo-Total-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
clin-tipoAlcohol,0.000000,0.000000,0.500000,5,0.000000,0.000000
clin-añosSinFumar,0.000000,0.000000,0.500000,5,0.000000,0.000000
